# Assignment 2
In this task a cancer cell detection system will be created.

Luke Mason - s3630120
Shekhar Hazari

#### Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler

#### Initialise helper functions¶

#### Unzip zipped data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change according to your directory structure
!ls /content/drive/'My Drive'/Collab Notebooks/ML-A2/

In [ ]:
# Change according to your directory structure
!cp /content/drive/'My Drive'/Collab Notebooks/ML-A2/Image_classification_data.zip .
!unzip -q -o Image_classification_data.zip
!rm Image_classification_data.zip
!ls

In [6]:
# Uncomment if on local machine or AWS
import zipfile
import os

if (not os.path.exists("./data_labels_maindata.csv")):
    with zipfile.ZipFile('./Image_classification_data.zip', 'r') as zip_ref:
        zip_ref.extractall('./')

#### Load the main dataset
This data contains both the cell type and isCancerous labels.

In [8]:
raw_dataframe_main = pd.read_csv('./data_labels_maindata.csv', delimiter=',')
raw_dataframe_main.describe()

,InstanceID,patientID,cellType,isCancerous
count,9896.000000,9896.000000,9896.000000,9896.000000
mean,10193.880154,29.762025,1.501516,0.412187
std,6652.912660,17.486553,0.954867,0.492253
min,1.000000,1.000000,0.000000,0.000000
25%,4135.750000,14.000000,1.000000,0.000000
50%,9279.500000,26.000000,2.000000,0.000000
75%,16821.250000,47.000000,2.000000,1.000000
max,22444.000000,60.000000,3.000000,1.000000


#### Load the extra dataset
This dataset does not contain the cellType labels.

In [9]:
raw_dataframe_extra = pd.read_csv('./data_labels_extraData.csv', delimiter=',')
raw_dataframe_extra.describe()

,InstanceID,patientID,isCancerous
count,10384.000000,10384.00000,10384.000000
mean,12087.866333,80.38203,0.287943
std,6173.866838,9.40388,0.452826
min,1631.000000,61.00000,0.000000
25%,6655.750000,71.00000,0.000000
50%,12377.500000,81.00000,0.000000
75%,16374.250000,88.00000,1.000000
max,22235.000000,99.00000,1.000000


### Data Setup
ImageName, InstanceID, features are excluded because they won't help in EDA since they are all unique values and are not information that is useful by themselves.

The raw data frame will be used to get the image names to map them to the images for the feature gathering.

In [20]:
# Construct non-target and target datasets. Y is target labels, X is non-target.
df_main_imageNames = raw_dataframe_main.drop(['patientID', 'InstanceID', 'isCancerous'], axis=1)
df_main_patients = raw_dataframe_main.drop(['ImageName', 'InstanceID', 'isCancerous'], axis=1)
df_main_target_labels = raw_dataframe_main['isCancerous']

In [21]:
# Construct non-target and target datasets. Y is target labels, X is non-target.
df_extra_imageNames = raw_dataframe_extra.drop(['patientID', 'InstanceID', 'isCancerous'], axis=1)
df_extra_patients = raw_dataframe_extra.drop(['ImageName', 'InstanceID', 'isCancerous'], axis=1)
df_extra_target_labels = raw_dataframe_extra['isCancerous']

### 1 - Exploratory Data Analysis
The data is assumed to be made of entirely by newborn babies according to the assignment forums.\

#### 1.1 - Exploring the distribution of the data
In this section the data's distribution of values are examined. Observations and folowwing conclusions are made to be used for justifying the decisions made in the rest of the report.

##### 1.1.1 Exploring unique values and data types
Here the fields display the unique values there are and their frequency.

In [22]:
for field in df_main_X.columns:
    df = raw_dataframe_main.groupby(field)['InstanceID'].nunique()
    print(df)
print(raw_dataframe_main.shape)

patientID
1      19
2      33
3     136
4     127
5     169
6     198
7     253
8     332
9     348
10    302
11     56
12    130
13    180
14    207
15    125
16    111
17    310
18    320
19    158
20    325
21    224
22    152
23    254
24    192
25    180
26    157
27     17
28     15
29    355
30    110
31    137
32     99
33    163
34     14
35     11
36    128
37     71
38     84
39    105
40    209
41    250
42    136
43    137
44    121
45     74
46    120
47    133
48    147
49    187
50    195
51    286
52    178
53    132
54    389
55    263
56     92
57    149
58    161
59    115
60    115
Name: InstanceID, dtype: int64
(9896, 6)


In [10]:
train_df_main, test_df_main = train_test_split(raw_dataframe_main, test_size=0.2, random_state=47)
train_df_main, val_df_main = train_test_split(train_df_main, test_size=0.25, random_state=47)

print("Train data : {}, Val Data: {}, Test Data: {}".format(train_df_main.shape[0], val_df_main.shape[0], test_df_main.shape[0]))

Train data : 5937, Val Data: 1979, Test Data: 1980


In [11]:
train_df_main['Class'] = train_df_main['Class'].astype('str')
val_df_main['Class'] = val_df_main['Class'].astype('str')

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df_main,
        directory='./',
        x_col="ImgPath",
        y_col="Class",
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_df_main.flow_from_dataframe(
        dataframe=val_data,
        directory='./',
        x_col="ImgPath",
        y_col="Class",
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')

KeyError: 'Class'